In [1]:
#полный пайплайн TEA
#изготавливает модель, сохраняет, разбирает, генерит с помощью модели

In [2]:
install = False
#Через requirements поставить это затруднительно, последовательность установки важна, так как часть либ компилируются
#Эта последовательность установки, как бы странно она ни выглядела, работает. Не в любых окружениях, но во многих.
if install:

    !pip install deepspeed==0.14.4

    !pip install --upgrade pip
    !pip install --upgrade transformers==4.46.2 tyro==0.9.8 triton==2.3.1 trl==0.12.0

    !pip install -v "xformers==0.0.29.post1"
    #!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
    !pip install --no-deps -v "peft==0.13.2"
    !pip install --no-deps packaging ninja einops flash-attn bitsandbytes==0.44.1
    !pip install accelerate==0.34.2
    !pip uninstall peft -y
    !pip install -v "peft==0.13.2"

In [3]:
import sys
import time
import json
import os
from collections import OrderedDict
from pathlib import Path
from collections import deque
import os, pickle, random
import psutil
import warnings
import codecs
import numpy as np
import pandas as pd
import requests
from typing import Dict, List, Optional
import torch.nn as nn
import torch.nn.functional as F

import torch
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader

import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModel
from transformers import StoppingCriteria, StoppingCriteriaList
from torch import cuda, LongTensor, FloatTensor
from peft import PeftModel, PeftConfig, PeftModelForCausalLM

import ensembles
from sequential_models import GPTAdapterLayer, GPTAdapterLayerWide, assemble_model, disassemble_model
from benchmarks import LLMBenchmark

In [4]:
current_device = torch.cuda.current_device()
print(f"Используется GPU с номером: {current_device}")

Используется GPU с номером: 0


In [5]:
device = torch.device("cuda")

In [6]:
print(f'Remaining GPU memory: {torch.cuda.memory_allocated() / (1024 ** 3):.2f} GB')

Remaining GPU memory: 0.00 GB


In [7]:
# Проверяем доступность CUDA и количество GPU
#если надо, можем переключиться на другую GPU
if 0:
    if torch.cuda.is_available():
        if torch.cuda.device_count() > 1:  # Если есть более 1 GPU
            device = torch.device("cuda:1")  # Явно указываем устройство 1
            print(f"Переключились на GPU 1")
        else:
            print("Доступен только один GPU (номер 0), переключение невозможно")
    else:
        print("CUDA недоступно")

In [8]:
start_train = False#Запускаем ли мы сейчас обучение с нуля, или с чекпоинта
learnable_linear_model = True#учим ли мы линейную субмодель внутри lm_head


cardinality = 128256#размер словаря токенов

padding_token = 128009
forbidden_tokens_list = [padding_token]
seed = int(np.random.rand() * 1000000)#random seed for data sampling

bits_per_number = 4

In [9]:
max_batch_size4loss = 1024 * 3#1024 * 2#loss считаем батчами, во избежание out of memory error
head_max_batch_size = 1024#lm_head заускаем батчами, во избежание out of memory error

In [10]:
use_cp_for_head = True#если True, то мы более устойчивы к OOM, если false, то быстрее

In [11]:
#конфигурации моделек
mode = 'pretrain'#'finetune', 'pretrain'
model_version = "heavy_4"#"light_1"#"heavy_4"
transformer_adapter_params = {}
lm_head_adapter_params = {'cardinality':cardinality}
optimizer_params = {}
depth2freeze = 0
if model_version == "heavy_2":
    transformer_adapter_params['embed_dim'] = 4096
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"
    #трансформерный адаптер
    transformer_adapter_params['num_heads_tlayer'] = 16
    transformer_adapter_params['ff_dim'] = transformer_adapter_params['embed_dim']  * 4
    transformer_adapter_params['t_layers_count'] = 2
    transformer_adapter_params['layer_configs'] = [1024*4] * 2
    optimizer_params['batch_size'] = 1#почему-то распараллеливание ухудшает скорость, а так щдесь можно поставить и другое число
    optimizer_params['accum_batch'] = 50
    optimizer_params['lr'] = 1e-6#lr для LM_head
    optimizer_params['lr_transformer'] = optimizer_params['lr'] * 0.05#lr для трансформерного адаптера
    optimizer_params['transformer_update_rate'] = 0.5#0.2#доля случаев, когда через трансформерный адаптер реально идут градиенты. Влияет на быстродействие
    transformer_adapter_params['transformer_adapter_weight'] = 1e-3#идея в том, что со старта адаптерный слой должен влиять совсем чуть-чуть
    transformer_adapter_params['dropout'] = 0.1
    transformer_adapter_params['concat'] = False
    optimizer_params['opt_type'] = 'adam'

    #адаптер для LM-head
    lm_head_adapter_params['embedding_size'] = transformer_adapter_params['embed_dim']
    lm_head_adapter_params['heavy_head'] = True
    lm_head_adapter_params['net_dropout_rate'] = 0
    lm_head_adapter_params['individ_dropout_rate'] = 0.03
    lm_head_adapter_params['layer_configs_head'] = [2048] * 5 + [1024] * 5
    lm_head_adapter_params['sample_features'] = 1
    lm_head_adapter_params['composition_size'] = 1
    lm_head_adapter_params['memnet_params'] = {}
    lm_head_adapter_params['use_memnets'] = False



elif model_version == "heavy_3":
    transformer_adapter_params['embed_dim'] = 4096
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"
    #трансформерный адаптер
    transformer_adapter_params['num_heads_tlayer'] = 32
    transformer_adapter_params['ff_dim'] = transformer_adapter_params['embed_dim']  * 2
    transformer_adapter_params['t_layers_count'] = 1
    transformer_adapter_params['layer_configs'] = [1024*4] * 4
    optimizer_params['batch_size'] = 1#почему-то распараллеливание ухудшает скорость, а так щдесь можно поставить и другое число
    optimizer_params['accum_batch'] = 13000
    optimizer_params['lr'] = 1e-6#lr для LM_head
    optimizer_params['transformer_update_rate'] = 0.5#0.2#доля случаев, когда через трансформерный адаптер реально идут градиенты. Влияет на быстродействие
    optimizer_params['lr_transformer'] = optimizer_params['lr'] * 0.1 * optimizer_params['transformer_update_rate']#lr для трансформерного адаптера
    transformer_adapter_params['transformer_adapter_weight'] = 1e-3#идея в том, что со старта адаптерный слой должен влиять совсем чуть-чуть
    transformer_adapter_params['dropout'] = 0.1
    transformer_adapter_params['concat'] = True
    optimizer_params['opt_type'] = 'adam'

    #адаптер для LM-head
    lm_head_adapter_params['embedding_size'] = transformer_adapter_params['embed_dim']
    lm_head_adapter_params['heavy_head'] = True
    lm_head_adapter_params['net_dropout_rate'] = 0
    lm_head_adapter_params['individ_dropout_rate'] = 0.03
    lm_head_adapter_params['layer_configs_head'] = [2048] * 6 + [1024] * 5
    lm_head_adapter_params['sample_features'] = 1
    lm_head_adapter_params['composition_size'] = 1
    lm_head_adapter_params['memnet_params'] = {}
    lm_head_adapter_params['use_memnets'] = False

elif model_version == "heavy_4":
    transformer_adapter_params['embed_dim'] = 4096
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"
    #трансформерный адаптер
    transformer_adapter_params['num_heads_tlayer'] = 32
    transformer_adapter_params['ff_dim'] = transformer_adapter_params['embed_dim']  * 2
    transformer_adapter_params['t_layers_count'] = 1
    transformer_adapter_params['layer_configs'] = [1024*4] * 10
    optimizer_params['batch_size'] = 1#почему-то распараллеливание ухудшает скорость, а так щдесь можно поставить и другое число
    optimizer_params['accum_batch'] = 1000#23000#10000
    optimizer_params['lr'] = 1e-6#lr для LM_head
    optimizer_params['transformer_update_rate'] = 0.#0.2#доля случаев, когда через трансформерный адаптер реально идут градиенты. Влияет на быстродействие
    optimizer_params['lr_transformer'] = optimizer_params['lr'] * 0.1 * optimizer_params['transformer_update_rate']#lr для трансформерного адаптера
    
    transformer_adapter_params['transformer_adapter_weight'] = 1e-3#идея в том, что со старта адаптерный слой должен влиять совсем чуть-чуть
    transformer_adapter_params['dropout'] = 0.1
    transformer_adapter_params['concat'] = True
    optimizer_params['opt_type'] = 'adam'#'adam'
    
    #адаптер для LM-head
    lm_head_adapter_params['embedding_size'] = transformer_adapter_params['embed_dim']
    lm_head_adapter_params['heavy_head'] = True
    # lm_head_adapter_params['net_dropout_rate'] = 0.
    # lm_head_adapter_params['individ_dropout_rate'] = 0.03
    # lm_head_adapter_params['layer_configs_head'] = [1024 * 3]  + [1024 * 2] * 2 + [1024] * 4 + [512] * 3
    # lm_head_adapter_params['sample_features'] = 1.
    # lm_head_adapter_params['composition_size'] = 1


    lm_head_adapter_params['net_dropout_rate'] = 0.7
    lm_head_adapter_params['individ_dropout_rate'] = 0.03
    lm_head_adapter_params['layer_configs_head'] = [256] * 2 + [128] * 5  + [64] * 5
    lm_head_adapter_params['sample_features'] = 0.7
    lm_head_adapter_params['composition_size'] = 10

    
    lm_head_adapter_params['memnet_params'] = {}
    lm_head_adapter_params['use_memnets'] = False


    lm_head_adapter_params['recreate_lm_head'] = False
    

    depth2freeze = 0#32#28

elif model_version == "light_1":
    #размерность "родного" трансформера
    transformer_adapter_params['original_transformer_size'] = 2048
    #размерность адаптера! Может не совпадать с размерностью "родного" трансформера
    transformer_adapter_params['embed_dim'] = 2048
    model_name = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit"
    #трансформерный адаптер
    transformer_adapter_params['num_heads_tlayer'] = 32
    transformer_adapter_params['ff_dim'] = transformer_adapter_params['embed_dim']  * 2
    transformer_adapter_params['t_layers_count'] = 2
    transformer_adapter_params['layer_configs'] = [1024*2] * 10
    optimizer_params['batch_size'] = 1#почему-то распараллеливание ухудшает скорость, а так здесь можно поставить и другое число
    optimizer_params['accum_batch'] = 100#10000
    optimizer_params['lr'] = 1e-6#lr для LM_head
    optimizer_params['transformer_update_rate'] = 1.#0.2#доля случаев, когда через трансформерный адаптер реально идут градиенты. Влияет на быстродействие
    optimizer_params['lr_transformer'] = optimizer_params['lr'] * 0.1 * optimizer_params['transformer_update_rate']#lr для трансформерного адаптера
    
    transformer_adapter_params['transformer_adapter_weight'] = 1e-3#идея в том, что со старта адаптерный слой должен влиять совсем чуть-чуть
    transformer_adapter_params['dropout'] = 0.1
    transformer_adapter_params['concat'] = True
    transformer_adapter_params['wide'] = True

    optimizer_params['opt_type'] = 'adam'#'adam'

    #адаптер для LM-head 
    lm_head_adapter_params['heavy_head'] = True
    #Ну типа исходный трансформер + все слои адаптера через конкат
    lm_head_adapter_params['embedding_size'] = transformer_adapter_params['embed_dim'] * transformer_adapter_params['t_layers_count'] + transformer_adapter_params['original_transformer_size']
    lm_head_adapter_params['net_dropout_rate'] = 0
    lm_head_adapter_params['individ_dropout_rate'] = 0.03
    lm_head_adapter_params['layer_configs_head'] = [1024 * 3] * 4 + [1024 * 2] * 2 + [1024 * 1] * 8
    lm_head_adapter_params['sample_features'] = 1
    lm_head_adapter_params['composition_size'] = 1
    lm_head_adapter_params['memnet_params'] = {}
    lm_head_adapter_params['use_memnets'] = False

    depth2freeze = 0


elif model_version == "light_2":
    transformer_adapter_params['embed_dim'] = 2048
    model_name = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit"
    #трансформерный адаптер
    transformer_adapter_params['num_heads_tlayer'] = 32
    transformer_adapter_params['ff_dim'] = transformer_adapter_params['embed_dim']  * 2
    transformer_adapter_params['t_layers_count'] = 2
    transformer_adapter_params['layer_configs'] = [1024*4] * 8
    optimizer_params['batch_size'] = 1#почему-то распараллеливание ухудшает скорость, а так щдесь можно поставить и другое число
    optimizer_params['accum_batch'] = 1000#10000
    optimizer_params['lr'] = 1e-6#lr для LM_head
    optimizer_params['transformer_update_rate'] = 0.#0.2#доля случаев, когда через трансформерный адаптер реально идут градиенты. Влияет на быстродействие
    optimizer_params['lr_transformer'] = optimizer_params['lr'] * 0.1 * optimizer_params['transformer_update_rate']#lr для трансформерного адаптера
    
    transformer_adapter_params['transformer_adapter_weight'] = 1e-4#идея в том, что со старта адаптерный слой должен влиять совсем чуть-чуть
    transformer_adapter_params['dropout'] = 0.1
    transformer_adapter_params['concat'] = True
    optimizer_params['opt_type'] = 'adam'#'adam'

    #адаптер для LM-head
    lm_head_adapter_params['embedding_size'] = transformer_adapter_params['embed_dim']
    lm_head_adapter_params['heavy_head'] = True
    lm_head_adapter_params['net_dropout_rate'] = 0
    lm_head_adapter_params['individ_dropout_rate'] = 0.03
    lm_head_adapter_params['layer_configs_head'] = [1024 * 4] * 7 + [1024 * 2] * 8
    lm_head_adapter_params['sample_features'] = 1
    lm_head_adapter_params['composition_size'] = 1
    lm_head_adapter_params['memnet_params'] = {}
    lm_head_adapter_params['use_memnets'] = False

In [12]:
lm_head_adapter_params['head_max_batch_size'] = head_max_batch_size
lm_head_adapter_params['learnable_linear_model'] = learnable_linear_model

In [13]:
path2tadapter = f"tadapter_{model_version}.pth"
path2lmhead = f"lmhead_{model_version}.pth"
tokenizer_name = model_name

In [14]:
#настройки датасета

max_loss_len = 3500#столько токенов можно пихать в Y, не больше
if mode == 'finetune':
    #претрейн
    max_seq_len4inference = 6850#в X и в Y можно пихать столько токенов, в каждый. Не больше.
else:
    #дообучение
    max_seq_len4inference = 6850
    #max_seq_len4inference = 5050
    #max_seq_len4inference = 2050

In [15]:
#Загрузили исходную модельку
tokenizer = AutoTokenizer.from_pretrained(model_name)#, cache_dir="D:\cache\huggingface\\"+ model_name)

if bits_per_number == 4:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4"#, bnb_4bit_compute_dtype=torch.bfloat16
    )
elif bits_per_number == 8:
    bnb_config = BitsAndBytesConfig(
        load_in_8bit=True, bnb_8bit_use_double_quant=True, bnb_8bit_quant_type="nf8"#, bnb_8bit_compute_dtype=torch.bfloat16
    )
else:
    bnb_config = None

tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
              model_name,
              quantization_config=bnb_config,
              cache_dir="D:\cache\huggingface\\"+ model_name)


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


In [16]:
#model.lm_head
torch.save(model.lm_head.weight, "lin_model.pth")

In [17]:
class InstructDatasetR(Dataset):
    def __init__(self, data_file, tokenizer, max_seq_length=1000000, cut=None):
        self.data_file = data_file
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length
        self.cut = cut
        self.data = self.load_data()
        self.log_samples = deque(maxlen=45)

    def load_data(self):
        with open(self.data_file, 'rb') as f:
            data = pickle.load(f)
        random.seed(seed)
        data = random.sample(data, len(data))
        
        # if self.cut is None:
        #     data = [data_cur for data_cur in data if len(data_cur[1])>5]
        # else:
        #     data = [data_cur for data_cur in data[:self.cut] if self.cut>5]
        # for i in range(100):
        #     print('***', data[i])
        #     try:
        #         if (not 'Алис' in data[i][0]) and (not 'Элеон"' in data[i][0]) and (not 'Софи"' in data[i][0]) and (not 'Сэм"' in data[i][0]) (not 'Алис' in data[i][1]) and (not 'Элеон"' in data[i][1]) and (not 'Софи"' in data[i][1]) and (not 'Сэм"' in data[i][1]):
        #             print('***', data[i])
        #     except Exception:
        #         pass
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        self.log_samples.append(self.data[idx])
        parts = self.data[idx]
        text = parts[0]
        label = parts[1]
        parts = [parts[0], parts[1], 1]
        for r_variant in [-2,-1,-0.5,0.5,1,2]:
            s = f"<r{r_variant}>"
            if s in label:
                parts[-1] = r_variant
                label = label.replace(s, '')
                break
        r = parts[-1]
        if text is None:
            text = label
        
        # Кодируем текст и метку с помощью tokenizer
        encoding = self.tokenizer.encode_plus(
            text,
            max_length=self.max_seq_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        input_ids = encoding['input_ids']
        attention_mask = encoding['attention_mask']
        
        # Кодируем метку
        label_encoding = self.tokenizer.encode_plus(
            label,
            max_length=self.max_seq_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        label_ids = label_encoding['input_ids']
        return {
            'input_ids': input_ids[0],
            'attention_mask': attention_mask,
            'labels': label_ids[0],
            'mult': torch.tensor(r)
        }

In [18]:
# Создание Dataset
#такая конструкция нужна на случай, что вы загружаете датасет на диск, сейчас час ночи, 
#датасет догрузится примерно через час, а завтра вам рано вставать
#с таким подходм код будет выполняться, когда найдёт на диске датасет
while 1:
    try:
        if mode == 'pretrain':
            #dataset = InstructDatasetR("./data/dataset_llm_full_mflat.pkl", tokenizer, max_seq_len, cut=None)
            dataset = InstructDatasetR("../data_tea/dataset_llm_full_flat.pkl", tokenizer, max_seq_length=1000000, cut=None)
        else:
            dataset = InstructDatasetR("../data_tea/dataset_llm_full_instruct.pkl", tokenizer, max_seq_length=1000000, cut=None)
        break
    except Exception as e:
        print(e, pd.Timestamp.now())
        time.sleep(10)
len(dataset)

1155497

In [19]:
def soft_clip(tens, mn, mx):
    mn = torch.tensor(mn).to(tens.device)
    mx = torch.tensor(mx).to(tens.device)
    tens[tens<mn] = torch.nn.Sigmoid()(tens[tens<mn]) + mn
    tens[tens>mx] = torch.nn.Sigmoid()(tens[tens>mx]) + mx
    return tens
# Функция для подсчёта количества последних токенов, равных padding_token
def count_padding(tensor, padding_token):
    counts = []
    for row in tensor:
        count = 0
        for token in reversed(row):
            if token.item() == padding_token:
                count += 1
            else:
                break
        counts.append(count)
    return min(counts)
def count_trainable_parameters(model: nn.Module) -> tuple[int, int]:
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    frozen = sum(p.numel() for p in model.parameters() if not p.requires_grad)
    return trainable, frozen
loss_fct = nn.CrossEntropyLoss(reduction = 'none')

In [20]:
#теперь сохранение адаптеров
def save_adapters(head, tadapter, path2lmhead, path2tadapter):
    head.training = False
    torch.save(head, path2lmhead)
    torch.save(tadapter, path2tadapter)
    head.training = True

In [21]:
head, tadapter, optimizer = assemble_model(model, 
                path2lmhead, 
                path2tadapter, 
                start_train=start_train,
                to_generate=False,
                lm_head_adapter_params=lm_head_adapter_params,
                transformer_adapter_params=transformer_adapter_params,
                optimizer_params=optimizer_params
              )
if optimizer_params['transformer_update_rate'] == 0.:
    tadapter.transformer_float_mode = 16
    tadapter.half()

loading tadapter
loading heavy LM head
Head. Обучаемые: 2,433,850,752
Замороженные: 0
Transformer adapter. Обучаемые: 0
Замороженные: 373,848,393


In [22]:
#заморозить первые N слоёв выходного резнета
print('resnet depth', len(head.submodels[0].layers), 'depth2freeze', depth2freeze)
for i in range(len(head.submodels) - 1):
    for l_num in range(depth2freeze):
        for p in head.submodels[i].layers[l_num].parameters():
            p.requires_grad = False

# for l_num in range(depth2freeze + 2):
#     for p in head.submodels[0].layers[l_num].parameters():
#         print(p.requires_grad)

resnet depth 52 depth2freeze 0


In [23]:
#head.submodels[0].layers[-12]

In [24]:
head.submodels[0].layers#[-4]

ModuleList(
  (0): Linear(in_features=5735, out_features=256, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Dropout(p=0.03, inplace=False)
  (3): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (4): Linear(in_features=256, out_features=256, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Dropout(p=0.03, inplace=False)
  (7): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (8): Linear(in_features=256, out_features=128, bias=True)
  (9): LeakyReLU(negative_slope=0.01)
  (10): Dropout(p=0.03, inplace=False)
  (11): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  (12): Linear(in_features=128, out_features=128, bias=True)
  (13): LeakyReLU(negative_slope=0.01)
  (14): Dropout(p=0.03, inplace=False)
  (15): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  (16): Linear(in_features=128, out_features=128, bias=True)
  (17): LeakyReLU(negative_slope=0.01)
  (18): Dropout(p=0.03, inplace=False)
  (19): LayerNorm((128,), eps=1e-05, elementwise_affin

In [25]:
proba_drop_idx = 0.04
lambda_l2 = 1e-5

In [26]:
#снять метрики
if not start_train:
    assemble_model(model, 
        path2lmhead, 
        path2tadapter, 
        start_train=False,
        to_generate=True,
        lm_head_adapter_params=lm_head_adapter_params,
        transformer_adapter_params=transformer_adapter_params,
        optimizer_params=optimizer_params
      )
    current_benchmark = LLMBenchmark()
    results = current_benchmark.run(model, tokenizer, device)
    disassemble_model(model)
    print(results)
#{'fact_score': 0.109, 'translation_rouge': 0.33223}

loading tadapter
loading heavy LM head


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


time 0 days 00:02:18.386420
{'fact_score': 0.174, 'translation_rouge': 0.25744, 'reasoning_score': 0.0}


In [27]:
print(pd.Timestamp.now())

2025-05-22 12:00:32.449286


In [ ]:
#TRAIN!!!
#Если вы задаётесь вопросом, почему такой странный цикл обучения, давайте посмотрим, что не сработало в "нормальных" циклах обучения.
#Можно было запихнуть трансформерный адаптер и lm-resnet в исходную LLM, разморозить их и заморозить всё остальное. 
#НО тогда по непонятной причине выходила крайне плохая точность обучения. loss = 1.65 на датасете, где у классического TEA (ну, чисто резнеты) loss = 0.85
#Две недели поисков выявили, что видимое различие в том, что в одном случае адаптер обучает ВНУТРИ сетки, а в другом СНАРУЖИ. Снаружи ПОЧЕМУ-ТО лучше
#
#Если проинференсить трансформерный адаптер, а потом LM-head, то запросто может получиться, что на этапе backward мы сдохнем по памяти.
#Как фиксить? Делаем, чтобы для всего множества эмбеддингов от трансформерной части loss считался батчами
#В таком случае возникает ошибка, мол, чё нам делать с градиентами в трансформерном адаптере - мы же их уже посчитали, а ты предлагаешь считать ещё раз!
#Решаем так: берёт сабсемпл от эмбеддингов трансформерного адаптера и считаем loss на сабсемпле
#Это не вполне честно, так как мы менее тщательно учитываем длинные последовательности, но эй, это работает!

print('batch_size', optimizer_params['batch_size'])
i_pointer = 0

batch_accum_counter = 0
loss_array = []
metr_array = []

log = []
while 1: 
    input_ids_full = []
    labels_full = []
    weights_full = []
    i = 0
    offset_counter = 0
    while i < optimizer_params['batch_size']:
        offset_counter += 1
        sample_cur  = dataset[i_pointer + i]
        input_ids = sample_cur['input_ids']
        labels = sample_cur['labels']
        weights = sample_cur['mult']
        
        input_ids_len = len(input_ids)
        labels_len = len(labels)
        input_ids = input_ids[input_ids!=padding_token]
        
        drop_idx = torch.rand_like(input_ids.to(torch.float)) < proba_drop_idx
        input_ids = input_ids[~drop_idx]
        
        labels = labels[labels!=padding_token]
        padding_size = input_ids_len + labels_len - len(input_ids) - len(labels)
        input_ids_cur = torch.cat([input_ids, labels, torch.zeros(padding_size, dtype=torch.int32) + padding_token])[:max_seq_len4inference]
        #print('input_ids', input_ids.shape, 'labels', labels.shape)
        labels_cur = torch.cat([input_ids * 0 + padding_token, labels, torch.zeros(padding_size, dtype=torch.int32) + padding_token])[:max_seq_len4inference]
        if torch.all(labels_cur) == padding_token:
            print('empty label')
            continue
        i += 1
        
        input_ids_full.append(input_ids_cur[:-1])
        labels_full.append(labels_cur[1:])
        weights_full.append(weights)
        #log += [labels_cur.shape[0], input_ids_cur.shape[0]]

    i_pointer += i #should be batch_size
    with torch.no_grad():
        input_ids_full = torch.stack(input_ids_full).to('cuda')
        labels_full = torch.stack(labels_full).to('cuda')
        weights_full = torch.stack(weights_full).to('cuda').T

        cnt_pads = count_padding(labels_full, padding_token)
        if cnt_pads >= input_ids_full.shape[1] - 5:
            continue
        if cnt_pads > 0:
            input_ids_full = input_ids_full[:, :-cnt_pads]
            labels_full = labels_full[:, :-cnt_pads]
            
        outp = model.forward(input_ids_full, output_hidden_states=True, return_dict=True)
        #outp = model.forward(input_ids_full, output_hidden_states=False, return_dict=True)

        
        outp['states'] = outp['hidden_states'][-1]
        outp['states'] = outp['states'][:, :input_ids_full.shape[1]].detach()
        #state_full = outp.last_hidden_state[:, :input_ids_full.shape[1]]
        state_full = outp['states']
        weights_full2d = torch.vstack([weights_full] * labels_full.shape[-1]).to('cuda').T
        labels_cur = labels_full.ravel()#.cpu().numpy()
        
        weights_cur = weights_full2d.ravel()

        del labels_full
        del weights_full2d
        del weights_full
        del outp
        idx = torch.isin(labels_cur, torch.tensor(forbidden_tokens_list).to(device))
        labels_cur = labels_cur[~idx]
        weights_cur = weights_cur[~idx]
    if state_full.shape[0] == 0:
        continue

    #Для скорости считаем градиенты через трансформер не всегда
    req_current = np.random.rand() < optimizer_params['transformer_update_rate']
    for p in tadapter.parameters():
        p.requires_grad = req_current

    state_full = tadapter(state_full)[0]
    state_cur = state_full.reshape([state_full.shape[0] * state_full.shape[1], state_full.shape[2]])
    state_cur = state_cur[~idx]
    if state_cur.shape[0] > max_loss_len:
        state_cur = state_cur[:max_loss_len]

    #ограничить размер
    
    #print('state_cur', state_cur.shape)
    del state_full
    perm = torch.randperm(state_cur.size(0), device=device)
    state_cur = state_cur[perm]
    weights_cur = weights_cur[perm]
    labels_cur = labels_cur[perm]

    loss = None
    for x_start_pointer in range(0, state_cur.shape[0], max_batch_size4loss):
        #logits = head(state_cur.to(torch.float32)[x_start_pointer: x_start_pointer + max_batch_size4loss])
        if use_cp_for_head:
            logits = torch.utils.checkpoint.checkpoint(head, state_cur.to(torch.float32)[x_start_pointer: x_start_pointer + max_batch_size4loss])
        else:
            logits = head(state_cur.to(torch.float32)[x_start_pointer: x_start_pointer + max_batch_size4loss])
        ce_loss = loss_fct(logits, labels_cur[x_start_pointer: x_start_pointer + max_batch_size4loss].view(-1))
        ce_loss = ce_loss * weights_cur[x_start_pointer: x_start_pointer + max_batch_size4loss].to('cuda')
        ce_loss = torch.clamp(ce_loss, -0.9, 10.0)#soft_clip(ce_loss, -0.6, 10.)
        loss = torch.mean(ce_loss)

        #l2 регуляризация
        l2_reg = torch.tensor(0.0, device=device)
        for param in tadapter.parameters():
            if param.requires_grad:
                l2_reg += torch.norm(param)
        for param in head.parameters():
            if param.requires_grad:
                l2_reg += torch.norm(param)
        #l2_reg = sum(p.norm(2) ** 2 for p in chain(tadapter.parameters(), head.parameters()) / 1e6)
                     
        total_loss = loss + lambda_l2 * l2_reg
        
        total_loss.backward()
        # Клиппинг градиентов: обрезает норму градиентов до 2.0
        #torch.nn.utils.clip_grad_norm_(tadapter.parameters(), max_norm=2.0 * optimizer_params['accum_batch'])
        #torch.nn.utils.clip_grad_norm_(head.parameters(), max_norm=2.0 * optimizer_params['accum_batch'])
        
        # if state_cur.shape[0] - (x_start_pointer + max_batch_size4loss) > 0:
        #     print('cannot backward, shape', state_cur.shape[0] - (x_start_pointer + max_batch_size4loss))
        break
       
    del state_cur
    del weights_cur
    

    acc = torch.mean((torch.argmax(logits, axis=-1) == labels_cur[x_start_pointer: x_start_pointer + max_batch_size4loss]).to(torch.float16))
    acc_copy = acc.cpu().numpy()
    del logits
    del labels_cur
    batch_accum_counter += 1
    loss_array += [torch.mean(ce_loss).item()]
    metr_array += [acc.item()]
    if batch_accum_counter >= optimizer_params['accum_batch'] :
        optimizer.step()
        optimizer.zero_grad()
        torch.cuda.empty_cache()
        print('ce_loss', np.round(np.mean(loss_array), 3), i_pointer, 'from', len(dataset), np.round(100 * i_pointer/len(dataset), 5), '%', 'acc', np.mean(metr_array), pd.Timestamp.now())
        loss_array = []
        metr_array = []
        batch_accum_counter = 0
        if (np.random.rand() < 1 and optimizer_params['accum_batch'] > 500) or (np.random.rand() < 0.1 and optimizer_params['accum_batch']  <= 500):
            try:
                save_adapters(head, tadapter, path2lmhead, path2tadapter)
                if np.random.rand()<0.1:
                    save_adapters(head, tadapter, path2lmhead + '.back', path2tadapter + '.back')

            except Exception:
                #Я не хочу, чтобы скрипт падал только потому, что кто-то завалил веь диск
                print("not saved")
            #снять метрики
            assemble_model(model, 
                path2lmhead, 
                path2tadapter, 
                start_train=False,
                to_generate=True,
                lm_head_adapter_params=lm_head_adapter_params,
                transformer_adapter_params=transformer_adapter_params,
                optimizer_params=optimizer_params
              )
            current_benchmark = LLMBenchmark()
            results = current_benchmark.run(model, tokenizer, device,4)
            disassemble_model(model)
            print(results)

batch_size 1


In [ ]:
#33 min за 1300 при batch_size=1
#45 min за 1300 при batch_size=2
#21 min за 1300 при batch_size=1, 'transformer_update_rate' = 0.1
#24 min за 1300 при batch_size=1, 'transformer_update_rate' = 0.2

In [ ]:
#если принудительно стопнули, можно добить батч
optimizer.step()
optimizer.zero_grad()
torch.cuda.empty_cache()
print(batch_accum_counter)
print('ce_loss', np.round(np.mean(loss_array), 3), i_pointer, 'from', len(dataset), np.round(100 * i_pointer/len(dataset), 5), '%', 'acc', np.mean(metr_array), pd.Timestamp.now())

In [ ]:
#ручной сейв
save_adapters(head, tadapter, path2lmhead, path2tadapter)

In [ ]:
#тестим модель
assemble_model(model, 
                path2lmhead, 
                path2tadapter, 
                start_train=False,
                to_generate=True,
                lm_head_adapter_params=lm_head_adapter_params,
                transformer_adapter_params=transformer_adapter_params,
                optimizer_params=optimizer_params
              )

In [ ]:
#prompt = 'Твоя роль: Сталин. \n Серёга: Какая фракция в игре Герои Меча и Магии 3 соответствует твоей личности? Your answer, ONLY ENGLISH (end by "|"):'
# prompt = ''''Шэдоухарт девушка сорока восьми лет. У неё черные волосы, заплетенные в длинную косу, и зеленые глаза, а справа на лице присутствует длинный шрам. Облачена в среднюю броню, вооружена булавой и круглым деревянным щитом. На голове девушка носит металлический обруч.

# Психологический профиль
# Шэдоухарт осторожна и не склонна раскрывать информацию о себе, своих мотивах и реликвии, которую она носит. С особым недоверием она'''
#prompt = 'Твоя роль: Девушка-флористка. \n Серёга: Какая фракция в игре Герои Меча и Магии 3 соответствует твоей личности? Your answer, ONLY ENGLISH (end by "|"):'
#prompt = 'Твоя роль: Data scientist. \n Серёга: Мне надо улучшить LLM. У меня ресурсы на запуск Llama 3.1 8B или Qwen 2.5 14B, квантованные до 4 бит. Мне нужно, чтобы мои модели былу умнее (на уровне 70 B), но при этом имели настолько же низкие требования по памяти и запускались так же быстро. Я готов обучать новую модель. Я готов пробовать другие архитектуры. Опиши план, что делать. Your answer, ONLY ENGLISH  (end by "|"):'
#prompt = 'Твоя роль: Инженер-ракетчик. \n Серёга: Мне надо сделать ракету, в домашних условиях, небольшую. Расскажи, как это провернуть? ТВОЙ ОТВЕТ НА РУССКОМ  (end by "|"):'
#prompt = 'Твоя роль: Геймер, студент-инженер. \n Серёга: Перечисли всех монстров из Doom, каких вспомнишь! Your answer, ONLY ENGLISH  (end by "|"):'
#prompt = 'Твоя роль: Гермиона Грейнджер. \n Серёга: Как справиться с василиском? Your answer, НА РУССКОМ (end by "|", 200 tokens):'
inputs = tokenizer(prompt, return_tensors="pt").to(device)
repetition_penalty = 1.2
t = pd.Timestamp.now()
generate_ids = model.generate(inputs.input_ids.to(device),
                                        #attention_mask=inputs["attention_mask"],
                                        top_k=200,
                                        top_p=0.9,
                                        max_new_tokens=200,
                                        temperature=0.01,
                                        pad_token_id=tokenizer.pad_token_id,
                                        eos_token_id=tokenizer.eos_token_id,
                                        bos_token_id=tokenizer.bos_token_id,
                                        #no_repeat_ngram_size=3,
                                        repetition_penalty=repetition_penalty,
                                        early_stopping=True,
                                        use_cache=True,
                                        num_beams=1,
                                        tokenizer=tokenizer)
print(pd.Timestamp.now() - t)
answer = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print(answer)